In [2]:
#Global Variables
IMAGES_SAVED_PATH = "image_data" # Folder to save different CAtegories of images
MAPPING_DIC = {
    "rock":0,
    "paper":1,
    "scissor":2,
    "none":3
}
NUMBER_OF_CATEGORIES = 4
input_shape=(250,250,3)

In [5]:
import cv2
import os
import numpy as np
import keras
from keras_applications.imagenet_utils import _obtain_input_shape 
from keras.models import Sequential
from keras_squeezenet import SqueezeNet
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.layers import Activation,Convolution2D,Dropout,GlobalAveragePooling2D

def mapper(x):
    return MAPPING_DIC[x]

images_dataset=[]
for directory in os.listdir(IMAGES_SAVED_PATH):
    path = os.path.join(IMAGES_SAVED_PATH,directory)
    if not os.path.isdir(path):
        continue
    for image in os.listdir(path):
        if image.startswith("."): #To bypass hidden files
            continue
        image_scanned = cv2.imread(os.path.join(path,image))
        image_scanned = cv2.cvtColor(image_scanned,cv2.COLOR_BGR2RGB)
        image_scanned = cv2.resize(image_scanned,(250,250))
        images_dataset.append([image_scanned,directory])

data,categories = zip(*images_dataset)
categories = list(map(mapper,categories))
categories = np_utils.to_categorical(categories)

model = Sequential([
        SqueezeNet(input_shape=input_shape,include_top=False),
        Dropout(0.2),
        Convolution2D(NUMBER_OF_CATEGORIES,(1,1),padding="same"),
        Activation("relu"),
        GlobalAveragePooling2D(),
        Activation("softmax")
    ])
model.compile(optimizer=Adam(lr=0.00005),loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(np.array(images_dataset),np.array(categories),epochs=10)
model.save("RockPaperScissors.h5")